In [1]:
from api.ml_logic.data_treatment import get_data, partition_data
from api.ml_logic.area_definition import define_area
import warnings
warnings.filterwarnings('ignore')

import datetime as dt
import pandas as pd
from prophet import Prophet


%load_ext autoreload
%autoreload 2

/home/matheus/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
big_frame = get_data()

tratando o big frame

In [29]:
X = big_frame[['latitude','longitude','hub']].dropna()

In [38]:
big_frame.drop(columns='hub',inplace=True)

In [39]:
big_frame.dropna(inplace=True)

In [21]:
from sklearn.neighbors import KNeighborsClassifier

In [31]:
X['hub'] = X['hub'].map({'Vila Olímpia':0, 'São Bernardo do Campo':1, 'São Cristóvão':2,
       'Alphaville':3, 'Tatuapé':4, 'Cabo Frio':5, 'Barra':6, 'Curitiba':7,
       'Vitória':8, 'Brasília':9, 'Recife':10, 'Campinas':11})

In [32]:
y = X['hub']
X = X.drop(columns='hub')

In [34]:
model = KNeighborsClassifier(n_neighbors=3)

In [35]:
model.fit(X,y)

KNeighborsClassifier(n_neighbors=3)

In [40]:
big_frame['hub']=model.predict(big_frame[['latitude','longitude']])

In [42]:
big_frame['hub'] = big_frame['hub'].map({0:'Vila Olímpia', 1:'São Bernardo do Campo', 2:'São Cristóvão',
       3:'Alphaville', 4:'Tatuapé', 5:'Cabo Frio', 6:'Barra', 7:'Curitiba',
       8:'Vitória', 9:'Brasília', 10:'Recife', 11:'Campinas'})

In [43]:
df = partition_data(big_frame,'São Cristóvão')

In [44]:
df = define_area(df,'São Cristóvão')

Início dos testes

In [45]:
df['event_date'] = pd.to_datetime(df['event_date']).dt.date

In [46]:
df = df[['event_date','area_id','user_id']].groupby(by=['event_date','area_id'],as_index=False).count()
df = df.rename(columns={'event_date': 'ds', 'user_id': 'y'})

In [82]:
max_date = df['ds'].max()

In [74]:
areas = df['area_id'].unique()

In [75]:
areas.sort()

In [76]:
areas

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [77]:
resposta = {'area': [], 'resultado': [], 'modelo': [], 'modelofuturo': []}

areas = df['area_id'].unique()
areas.sort()

for area in areas:
    
    prophet = Prophet()
    area_df = df[df['area_id']==area]
    area_df.drop(columns='area_id', inplace=True)
    saida_fit = prophet.fit(area_df)
    saida_futuro = prophet.make_future_dataframe(periods=20, freq='D', include_history=True)
    saida_predict = prophet.predict(saida_futuro)
    resposta['modelo'].append(prophet)
    resposta['modelofuturo'].append(saida_futuro)
    resposta['area'].append(area)
    resposta['resultado'].append(saida_predict)

11:41:02 - cmdstanpy - INFO - Chain [1] start processing
11:41:02 - cmdstanpy - INFO - Chain [1] done processing
11:41:03 - cmdstanpy - INFO - Chain [1] start processing
11:41:03 - cmdstanpy - INFO - Chain [1] done processing
11:41:04 - cmdstanpy - INFO - Chain [1] start processing
11:41:04 - cmdstanpy - INFO - Chain [1] done processing
11:41:05 - cmdstanpy - INFO - Chain [1] start processing
11:41:05 - cmdstanpy - INFO - Chain [1] done processing
11:41:05 - cmdstanpy - INFO - Chain [1] start processing
11:41:06 - cmdstanpy - INFO - Chain [1] done processing
11:41:06 - cmdstanpy - INFO - Chain [1] start processing
11:41:06 - cmdstanpy - INFO - Chain [1] done processing
11:41:07 - cmdstanpy - INFO - Chain [1] start processing
11:41:07 - cmdstanpy - INFO - Chain [1] done processing
11:41:08 - cmdstanpy - INFO - Chain [1] start processing
11:41:08 - cmdstanpy - INFO - Chain [1] done processing
11:41:09 - cmdstanpy - INFO - Chain [1] start processing
11:41:09 - cmdstanpy - INFO - Chain [1]

In [131]:
demmand = {'area':[],'value':[]}

for area in resposta['area']:
    df = resposta['resultado'][area]
    demmand['area'].append(area)
    demmand['value'].append(df.loc[df['ds'].dt.date==max_date]['yhat'].values[0])



In [142]:
pd.to_datetime('01/01/2022')

Timestamp('2022-01-01 00:00:00')

In [136]:
import numpy as np

In [138]:
np.array(demmand['value']).sum()

1827.8133061179326

In [141]:
teste.loc[teste['ds'].dt.date==max_date]['yhat'].values[0]

27.314297246440848

In [140]:
teste['yhat'].loc[teste['ds'].dt.date==max_date]

398    27.314297
Name: yhat, dtype: float64